# IMPORTAÇÃO DOS DADOS

**Importando as bibliotecas**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

**Importando o dataset**

Dados obtidos no portal da transparência do senado, pode ser obtido [clicando aqui](https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps).
>Ao importar os arquivos .csv, me deparei com o erro 'utf-8 codec can't decode byte 0xf3 in position 213: invalid continuation byte'.  
>Consegui resolver definindo o encoding para 'cp1252', não obtive sucesso com 'latin1' ou 'utf-8'.

In [ ]:
ceaps_18 = pd.read_csv('data/despesa_ceaps_2018.csv', sep=';', header=1, encoding='cp1252')
ceaps_19 = pd.read_csv('data/despesa_ceaps_2019.csv', sep=';', header=1, encoding='cp1252')
ceaps_20 = pd.read_csv('data/despesa_ceaps_2020.csv', sep=';', header=1, encoding='cp1252')
ceaps_21 = pd.read_csv('data/despesa_ceaps_2021.csv', sep=';', header=1, encoding='cp1252')

**Unindo os datasets**

In [ ]:
ceaps_unido = [ceaps_18, ceaps_19, ceaps_20, ceaps_21]
ceaps = pd.concat(ceaps_unido)
ceaps.reset_index(drop=True)
ceaps.head(3)

# ANÁLISE DOS DADOS

In [ ]:
# Verifica o formato e o tipo dos dados
ceaps.info()

**Verificando valores ausentes**

In [ ]:
# Verifica a quantidade de valores ausentes
ceaps.isna().sum()

In [ ]:
# Verifica a porcentagem de valores ausentes
print('Valores ausentes na coluna Documento:', round((3045/77043)*100,2),'%')
print('Valores ausentes na coluna Detalhamento:', round((28640/77043)*100,2),'%')

**Verificando valores ausentes como gráfico**

In [ ]:
# Agrupa as quantidade de dados nulos e não nulos em um dataframe
ceaps_isnull = ceaps.isnull().sum()
ceaps_notnull = ceaps.notnull().sum()
ceaps_colunas = pd.DataFrame({'Válido':ceaps_notnull, 'Ausente':ceaps_isnull})

# Gráfico de barras
ax = ceaps_colunas.sort_values(by='Ausente',ascending =False).plot.bar(stacked=True, width=0.7, figsize = (12,3))

pos = ax.get_position()
ax.set_position([pos.x0, pos.y0, pos.width * 0.7, pos.height])
ax.legend(loc='right', prop={'size': 8}, bbox_to_anchor=(1.15, 0.925))

plt.xticks(fontsize=6, rotation=30)
plt.yticks(fontsize=8, rotation=0)
ax.set_title('VALORES AUSENTES')

**Verificando valores ausentes como tabela**

In [ ]:
# Verificando visualmente os valores ausentes do campo DETALHAMENTO
ceaps[ceaps['DOCUMENTO'].isnull()].head()

In [ ]:
# Verificando visualmente os valores ausentes do campo DETALHAMENTO
ceaps[ceaps['DETALHAMENTO'].isnull()].head()
#ceaps[['TIPO_DESPESA', 'FORNECEDOR', 'DETALHAMENTO']][ceaps['DETALHAMENTO'].isna()]

# MANIPULAÇÃO DOS DADOS

**Preenchendo valores ausentes**

In [ ]:
ceaps['DETALHAMENTO'] = (
    ceaps['DETALHAMENTO'].fillna('N/D') # N/D = Não disponível
    .reset_index(drop=True)
)
ceaps = ceaps.drop(columns=['DOCUMENTO']) 

# ceaps['DOCUMENTO'] = (    
#     ceaps['DOCUMENTO'].fillna('S/N') #S/N = Sem número
#     .reset_index(drop=True)
#   )

In [ ]:
# Verifica se os valores ausente foram preenchidos
ceaps.loc[ceaps['DETALHAMENTO'].str.contains('N/D', na=False)].head(5)

In [ ]:
# Verifica se os valores ausente de DETALHAMENTO foram preenchidos
ceaps[['TIPO_DESPESA', 'FORNECEDOR', 'DETALHAMENTO']][ceaps['DETALHAMENTO'].isna()]

In [ ]:
# Visualiza se os valores ausentes de DETALHAMENTO estão preenchidos com N/D
ceaps.loc[ceaps['DETALHAMENTO'].str.contains('N/D', na=False)].head()

In [ ]:
# Visualiza se os valores ausentes de DOCUMENTO estão preenchidos com S/D
#ceaps.loc[ceaps['DOCUMENTO'].str.contains('S/N', na=False)].head()

**Formatando os dados**

In [ ]:
# Verifica os fornecedores com a palavra S/A

ceaps.loc[ceaps['FORNECEDOR'].str.contains('S/A', na=False)].drop_duplicates(subset=['FORNECEDOR'])

In [ ]:
ceaps.iloc[[40]]

In [ ]:
ceaps.FORNECEDOR.head()

In [ ]:
# Formata os dados
ceaps['SENADOR'] = ceaps['SENADOR'].str.title() #Senador com primeira letra maiúsculo
ceaps['FORNECEDOR'] = ceaps['FORNECEDOR'].str.title() #Fornecedor com primeira letra maiúsculo
ceaps['FORNECEDOR'] = ceaps['FORNECEDOR'].replace('Ltda','LTDA.', regex=True)
ceaps['FORNECEDOR'] = ceaps['FORNECEDOR'].replace(r'LTDA\..', 'LTDA.', regex=True)
ceaps['FORNECEDOR'] = ceaps['FORNECEDOR'].replace(' S A',' S.A.', regex=True)
ceaps.head(3)

#ceaps['DOCUMENTO'] = ceaps['DOCUMENTO'].replace('(?i)s\/n(.*)','S/N', regex=True) #Transforma s/n em S/N. (?i) ignora case-sensitive, s\/n busca por s/n, (.*) com qualquer caractere depois.

In [ ]:
# Verifica se DETALHAMENTO contém 'n/d' em maíusculo
ceaps.loc[ceaps['DETALHAMENTO'].str.contains('N/D', na=False)].drop_duplicates(subset=['DETALHAMENTO'])

In [ ]:
# Verifica se DOCUMENTO contém 'n/d' em maiúsculo
#ceaps.loc[ceaps['DOCUMENTO'].str.contains('S/N', na=False)].drop_duplicates(subset=['DOCUMENTO'])

In [ ]:
# Verifica se DOCUMENTO contém 's/n' em minúsculo
#ceaps.loc[ceaps['DOCUMENTO'].str.contains('s/n', na=False)].drop_duplicates(subset=['DOCUMENTO'])

In [ ]:
# Verifica se FORNECEDER contém 'LTDA'
ceaps.loc[ceaps['FORNECEDOR'].str.contains('LTDA', na=False)].drop_duplicates(subset=['FORNECEDOR'])

In [ ]:
# Verifica se FORNECEDER contém 'S.A.'
ceaps.loc[ceaps['FORNECEDOR'].str.contains('S.A.', na=False)].drop_duplicates(subset=['FORNECEDOR'])

In [ ]:
# Verifica se ainda resta valores ausentes
ceaps.isna().sum()

In [ ]:
# Verifica quais TIPO_DESPESA existem
ceaps[['TIPO_DESPESA']].drop_duplicates()

In [ ]:
# Renomeia o TIPO_DESPESA para ficar consistente ao canal de transparência dos senadores individuais
# Exemplo: https://www6g.senado.leg.br/transparencia/sen/5985/?ano=2022

ceaps['TIPO_DESPESA'] = ceaps['TIPO_DESPESA'].replace({'Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.':'Aluguel de imóveis para escritório político',
       'Divulgação da atividade parlamentar': 'Divulgação da atividade parlamentar',
       'Passagens aéreas, aquáticas e terrestres nacionais': 'Passagens aéreas, aquáticas e terrestres nacionais',
       'Locomoção, hospedagem, alimentação, combustíveis e lubrificantes': 'Locomoção, hospedagem, alimentação e combustíveis',
       'Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar': 'Contratação de serviços de apoio ao parlamentar ',
       'Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. ': 'Aquisição de material de consumo ',
       'Serviços de Segurança Privada': 'Serviços de Segurança Privada '})

In [ ]:
ceaps[['TIPO_DESPESA']].drop_duplicates()

**Convertendo os dados**

In [ ]:
ceaps.info()

In [ ]:
# Converte o VALOR_REEMBOLSADO para float
ceaps['VALOR_REEMBOLSADO'] = ceaps['VALOR_REEMBOLSADO'].str.replace(',', '.')
ceaps['VALOR_REEMBOLSADO'] = ceaps['VALOR_REEMBOLSADO'].apply(float)

# Para visualização dos valores de forma mais humana
# ceaps['VALOR_REEMBOLSADO'] = ceaps['VALOR_REEMBOLSADO'].map('{:,.2f}'.format).str.replace(',', '~').str.replace('.', ',').str.replace('~', '.')

ceaps.iloc[[40]]

In [ ]:
ceaps[['VALOR_REEMBOLSADO']].boxplot()

In [ ]:
plt.plot(ceaps.VALOR_REEMBOLSADO)

In [ ]:
ceaps[['SENADOR', 'VALOR_REEMBOLSADO', 'TIPO_DESPESA', 'DETALHAMENTO']].drop_duplicates(subset='VALOR_REEMBOLSADO').sort_values('VALOR_REEMBOLSADO').head(20)

In [ ]:
ceaps[ceaps.VALOR_REEMBOLSADO < 10].sort_values(by='VALOR_REEMBOLSADO')

In [ ]:
# Quantos valores abaixo de 10 reais existem
ceaps[ceaps.VALOR_REEMBOLSADO < 10].shape[0]

In [ ]:
# Quantos valores acima de 40 mil reais existem
ceaps[ceaps.VALOR_REEMBOLSADO > 40000].shape[0]

In [ ]:
ceaps['VALOR_REEMBOLSADO'].describe()

In [ ]:
# Verifica se o VALOR_REEMBOLSADO foi convertido para float
ceaps.info()

**Convertendo o valor data**

In [ ]:
# Separa o mês e ano e substitui o DATA_TEMP
ceaps['DATA_TEMP'] = ceaps['DATA']

# Converte para datetime (ano-mês-dia) e transforma os valores incorretos em NaT
# ceaps['DATA'] = pd.to_datetime(ceaps['DATA'], format='%Y-%m-%d', errors = 'coerce')
ceaps['DATA'] = pd.to_datetime(ceaps['DATA'], dayfirst=True, errors = 'coerce')

In [ ]:
# Verifica se ainda tem dados não convertidos para datetime
ceaps.isna().sum()

In [ ]:
# Organiza a data em ordem ascendente
ceaps['DATA'].sort_values()

In [ ]:
# Seleciona as datas não convertidas juntamente com a data antes da conversão para comparação
ceaps[['ANO', 'MES', 'DATA', 'DATA_TEMP']][ceaps['DATA'].isna()]

In [ ]:
# Filtra todas as datas menores que 2017
ceaps[pd.to_datetime(ceaps['DATA'].astype(str)).dt.year <= 2016].drop_duplicates()
# Notei que o campo ANO e MES possui inconsistências com o campo DATA
# Decidi aplicar o mesmo método de separar o dia, mês, ano e aplicar no campo DATA ao invés de fillna

In [ ]:
#ceaps['DIA'] = ceaps.DATA.str.slice(-2)
#ceaps['MES'] = ceaps.DATA.str.slice(5, 7)
#ceaps['MES'] = ceaps['MES'].astype('str').apply(lambda x: '0'+x if len(x)==1 else  x)

ceaps['DATA'] = ceaps['DATA'].astype('str') #transforma a data em string
ceaps['DIA_MES_ANO'] = ceaps['ANO'].astype('str') + '-' + ceaps.DATA.str.slice(5, 7) + '-' + ceaps.DATA.str.slice(-2) #une o ano+mês+dia
ceaps['DATA'] = ceaps['DIA_MES_ANO'] #substitui a DATA pelos valores corretos

ceaps['DATA'] = pd.to_datetime(ceaps['DATA']) #converte a data para datetime
ceaps = ceaps.drop(columns=['DIA_MES_ANO', 'DATA_TEMP']) #dropa as colunas temporárias

In [ ]:
ceaps.info()

In [ ]:
# Verifica se ainda existe valores ausentes
ceaps.isna().sum()